# 선분의 기울기를 이용하여 세부 추출

* 다른 이미지에 대하여 차선이 아닌 선분이 검출 될  수 있음
* 앞에서 추출한 선분에서 기울기 조건을 제한하여 세부 검출

In [2]:
import cv2
import numpy as np

def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold): # Canny알고리즘
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kerner_size): # 가우시안 필터
    return cv2.GaussianBlur(img, (kerner_size, kerner_size), 0)

def region_of_interest(img, vertices, color3 = (255, 255, 255), color1 = 255):
    #ROI setting
    mask = np.zeros_like(img) # img와 같은 크기의 빈 이미지

    if len(img.shape) > 2 : #if color image (3 channels)
        color = color3
    else : #if black&white image ( 1 channel)
        color = color1
    
    #vertices에 정한 점들로 이루어진 다각형 부분(ROI부분)을 color로 채움
    cv2.fillPoly(mask, vertices, color)
    
    #이미지와 color로 체워진 ROI를 합침
    ROI_img = cv2.bitwise_and(img, mask)
    return ROI_img

def draw_lines(img, lines, color=[0,0,255], thickness = 2): #선그리기
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): #허프 변환
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
    #draw_lines(line_img, lines)
    
    return lines

def weighted_img(img, initial_img, a = 1, b = 1., c = 0.): #두 이지미 합치기
    return cv2.addWeighted(initial_img, a, img, b, c)
   

In [3]:
from audioop import lin2lin


image = cv2.imread("slope_test.jpg")
    
height, width = image.shape[:2]

gray_img = grayscale(image) # 흑백이미지로 변환

blur_img = gaussian_blur(gray_img, 3) # Blur 효과

canny_img = canny(blur_img, 70, 210) # Canny Edge 알고리ㅑ

cv2.imshow('canny', canny_img) # Canny 이미지 출력

vertices = np.array([[(50, height), (width/2 - 45, height/2 + 60), (width/2 + 45, height/2 + 60), (width-50, height)]], dtype=np.int32)

roi_img = region_of_interest(canny_img, vertiwces) # vertices에 정한 점들을 기준으로 ROI이미지 생성
cv2.imshow('roi', roi_img)

line_arr = hough_lines(roi_img, 1, 1 * np.pi/180, 30, 10 ,20) #허프 변환
line_arr = np.squeeze(line_arr)

#기울기 구하기
slope_degree = (np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:, 2]) * 180) / np.pi

#수평 기울기 제한
line_arr = line_arr[np.abs(slope_degree) < 160]
slope_degree = slope_degree[np.abs(slope_degree) < 160]
#수직 기울기 제한
line_arr = line_arr[np.abs(slope_degree) > 95]
slope_degree = slope_degree[np.abs(slope_degree) > 95]

#필터링 직선 버리기
L_lines, R_lines = line_arr[(slope_degree > 0), :], line_arr[(slope_degree < 0), :]
temp = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
L_lines, R_lines = L_lines[:,None], R_lines[:,None]

draw_lines(temp, L_lines)
draw_lines(temp, R_lines)

result = weighted_img(temp, image)
cv2.imshow('result', result)
cv2.waitKey(0)
